In [3]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier

In [4]:
df = pd.read_csv("../all_data_Char.csv")

In [15]:
# create the test and the train data set. 
disordered = df.loc[df["label"]==1.0]
ordered = df.loc[df["label"]==0.0]
# select indices based on a random distributions 
disordered["is_train"] = np.random.uniform(0, 1, len(disordered)) <= 0.885 # calculated based on test/total
ordered["is_train"] = np.random.uniform(0,1, len(ordered)) <= 0.9947 # calculated based on test/total

/Users/michellegarcia/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/michellegarcia/miniconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,sequence,seq_len,molecular_weight,hydrophobicity,instability_index,aromaticity,label,is_train
0,MASREEEQRETTPERGRGAARRPPTMEDVSSPSPSPPPPRAPPKKR...,529,59138.7350,-0.611153,58.945936,0.058601,1.0,True
1,MKTQRDGHSLGRWSLVLLLLGLVMPLAIIAQVLSYKEAVLRAIDGI...,170,19301.1649,-0.441765,37.586471,0.064706,1.0,True
2,MDAQTRRRERRAEKQAQWKAANPLLVGVSAKPVNRPILSLNRKPKS...,107,12298.1162,-0.919626,46.321495,0.037383,1.0,True
3,MGDVEKGKKIFVQKCAQCHTVEKGGKHKTGPNLHGLFGRKTGQAPG...,105,11832.6058,-0.875238,15.507619,0.085714,1.0,True
4,MPKRGKKGAVAEDGDELRTEPEAKKSKTAAKKNDKEAAGEGPALYE...,318,35554.0730,-0.583019,43.411950,0.088050,1.0,True


In [46]:
df2 = pd.concat([disordered,ordered])
df2 = df2.drop(columns=["sequence"])
train, test = df2[df2['is_train']==True], df2[df2['is_train']==False]

In [43]:
print('Number of observations in the training data:', len(train))
print('Number of observations in the test data:',len(test))

Number of observations in the training data: 248883
Number of observations in the test data: 2535


In [50]:
features = df2.columns[1:5]
y = train.label.to_list()

In [81]:
clf = RandomForestClassifier(n_jobs=2, n_estimators =5, random_state=0)
clf.fit(train[features], y)
clf.predict(test[features])
clf.predict_proba(test[features])[0:10]

array([[1. , 0. ],
       [0. , 1. ],
       [1. , 0. ],
       [0. , 1. ],
       [1. , 0. ],
       [1. , 0. ],
       [0.4, 0.6],
       [0. , 1. ],
       [0.4, 0.6],
       [0. , 1. ]])

In [82]:
preds = clf.predict(test[features])

In [83]:
pd.crosstab(test['label'], preds, rownames=['Actual label'], colnames=['Predicted label'])

Predicted label,0.0,1.0
Actual label,,
0.0,1327,10
1.0,795,403


In [62]:
list(zip(train[features], clf.feature_importances_))

[('molecular_weight', 0.2948203424700344),
 ('hydrophobicity', 0.24947013914969063),
 ('instability_index', 0.25810770450472803),
 ('aromaticity', 0.19760181387554704)]